# Building an ETL

In [1]:
# Import our Libraries
import pandas as pd
import sqlite3

In [2]:
# Read in our CSV
df = pd.read_csv('client_orders.csv')

In [3]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Optionally handle missing values
df.fillna(0, inplace=True)  # Example: Fill missing values with 0

Missing values per column:
order number          0
employee id           0
employee name         0
employee name.1       0
employee_job_title    0
order date            0
order type            0
customer type         0
cust id               0
customer name         0
customer name.1       0
prod category         0
prod number           0
prod_name             0
quantity              0
price                 0
order total           0
dtype: int64


In [4]:
# Display column names
df.columns

Index(['order number', 'employee id', 'employee name', 'employee name.1',
       'employee_job_title', 'order date', 'order type', 'customer type',
       'cust id', 'customer name', 'customer name.1', 'prod category',
       'prod number', 'prod_name', 'quantity', 'price', 'order total'],
      dtype='object')

In [5]:
# Select specific columns relevant to the analysis
df_unique_cols = df[['order number', 'employee id', 'employee name',
   'employee_job_title', 'order date', 'order type', 'customer type',
   'cust id', 'customer name', 'prod category',
   'prod number', 'prod_name', 'quantity', 'price', 'order total']]

In [14]:
# Reset the index and clean column names (replacing spaces with underscores for consistency)
df_clean_cols = df_unique_cols.reset_index()
df_clean_cols.columns = df_clean_cols.columns.str.replace(' ', '_').str.lower()

In [7]:
df_clean_cols.head()

,index,order_number,employee_id,employee_name,employee_job_title,order_date,order_type,customer_type,cust_id,customer_name,prod_category,prod_number,prod_name,quantity,price,order_total
0,0,1102935,900019019,Alexandra Kundt,Senior Sales Associate,2/9/2019,Retail,Individual,1574,Skipton Fealty,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,$45.00,135.0
1,1,1102935,900019019,Alexandra Kundt,Senior Sales Associate,2/9/2019,Retail,Individual,1574,Skipton Fealty,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,$45.00,135.0
2,2,1102976,900019019,Alexandra Kundt,Senior Sales Associate,2/15/2019,Retail,Individual,1809,Lanni D'Ambrogi,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,$19.50,19.5
3,3,1102976,900019019,Alexandra Kundt,Senior Sales Associate,2/15/2019,Retail,Individual,1809,Lanni D'Ambrogi,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,$19.50,19.5
4,4,1102988,900019019,Alexandra Kundt,Senior Sales Associate,2/16/2019,Retail,Individual,2253,Far Pow,Olive Oil,OO302,Chili Extra Virgin Olive Oil,4,$26.00,104.0


In [8]:
# Check unique values for categorical columns
print("Unique values per column:")
print(df_clean_cols.nunique())

Unique values per column:
index                 5138
order_number          5130
employee_id             44
employee_name           46
employee_job_title       9
order_date             731
order_type               2
customer_type            2
cust_id               2417
customer_name         2417
prod_category            3
prod_number             67
prod_name               67
quantity                14
price                   21
order_total            187
dtype: int64


In [9]:
# Identify duplicate rows in the dataset to ensure data integrity before further processing.
df_clean_cols[df.duplicated()]

,index,order_number,employee_id,employee_name,employee_job_title,order_date,order_type,customer_type,cust_id,customer_name,prod_category,prod_number,prod_name,quantity,price,order_total
1,1,1102935,900019019,Alexandra Kundt,Senior Sales Associate,2/9/2019,Retail,Individual,1574,Skipton Fealty,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,$45.00,135.00
3,3,1102976,900019019,Alexandra Kundt,Senior Sales Associate,2/15/2019,Retail,Individual,1809,Lanni D'Ambrogi,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,$19.50,19.50
20,20,1103475,900019019,Alexandra Kundt,Senior Sales Associate,4/29/2019,Retail,Individual,3654,Gabriel Polini,Bath products,BP402,Assorted Olive Bar Soap - 6 pk,3,$30.00,90.00
106,106,1107133,900019019,Alexandra Kundt,Senior Sales Associate,10/2/2020,Retail,Individual,3050,Damian Murcutt,Bath products,BP301,Mandarin and Olive Bath Salts,3,$8.50,25.50
116,116,1107392,900019019,Alexandra Kundt,Senior Sales Associate,11/9/2020,Retail,Individual,1583,Georgianna Harrild,Bath products,BP305,Mandarin and Olive Oil Lotion,2,$9.99,19.98
129,129,1107872,900019019,Alexandra Kundt,Senior Sales Associate,1/14/2021,Retail,Individual,2863,Charmine Eyers,Olive Oil,OO202,Extra Virgin Olive Oil - Medium 2pk,3,$16.75,50.25


In [10]:
# Remove duplicate rows based on specific columns
df_clean_cols_rows = df_clean_cols.drop_duplicates(subset=['order_number', 'order_date'])

In [11]:
df_clean_cols.head()

,index,order_number,employee_id,employee_name,employee_job_title,order_date,order_type,customer_type,cust_id,customer_name,prod_category,prod_number,prod_name,quantity,price,order_total
0,0,1102935,900019019,Alexandra Kundt,Senior Sales Associate,2/9/2019,Retail,Individual,1574,Skipton Fealty,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,$45.00,135.0
1,1,1102935,900019019,Alexandra Kundt,Senior Sales Associate,2/9/2019,Retail,Individual,1574,Skipton Fealty,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,$45.00,135.0
2,2,1102976,900019019,Alexandra Kundt,Senior Sales Associate,2/15/2019,Retail,Individual,1809,Lanni D'Ambrogi,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,$19.50,19.5
3,3,1102976,900019019,Alexandra Kundt,Senior Sales Associate,2/15/2019,Retail,Individual,1809,Lanni D'Ambrogi,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,$19.50,19.5
4,4,1102988,900019019,Alexandra Kundt,Senior Sales Associate,2/16/2019,Retail,Individual,2253,Far Pow,Olive Oil,OO302,Chili Extra Virgin Olive Oil,4,$26.00,104.0


In [12]:
# Connect to SQLite database
conn = sqlite3.connect('database.db')

In [13]:
# Connect to SQLite database
df_clean_cols_rows.to_sql('client_orders_table', conn, if_exists='replace', index=False)

5130